In [9]:
%run ./"0 Imports and functions.ipynb"

/home/ben/miniconda3/bin/python
3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
astropy 5.1.dev153+gb740594dc
spectral_cube 0.6.1.dev22+g003ef16
spectral_cube file path /home/ben/.local/lib/python3.8/site-packages/spectral_cube/__init__.py
reproject 0.8
3.3.4 Matplotlib
/home/ben/miniconda3/lib/python3.8/site-packages/matplotlib/__init__.py
1.23.1 Numpy
astrodendro_file: /home/ben/.local/lib/python3.8/site-packages/astrodendro/__init__.py
Results will be saved to Directory ./Result Files
Cubes will be saved and loaded with Directory ./Spectral Cubes
Plots will be saved to Directory ./Plots
Dendrograms will be saved to Directory ./Dendrograms


In [10]:
# Pointing_Information

Pointing_Information = {}

#This is the stuff that needs changing between cubes

#Pointing_Information["Original_File_Name"] = "COGAL_deep_interp.fits"+"_Header_Fix.fits"
Pointing_Information["Original_File_Name"] ="12CO-2_BEARS.fits"
#"CHIMPS2_CMZ_12COall_beammatch_253_reproject_specinterp.matched.noise_matched.fits"
#"_cmz_CO_J3_2_Spliced_Smoothed_003-357.fits" #the name of the initial SC file.
File_Descriptor='cmz_CO_J1_0_high_res'
#File_Descriptor='cmz_CO_J1_0_COMPLETE'

Pointing_Information["File_Descriptor"] = File_Descriptor
Pointing_Information["target"] = "CMZ"#
Pointing_Information["center"] = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic') #the center of the cmz
Pointing_Information["crop_center"] = SkyCoord('00d00m00.00s  ', '-00d00m00.000s', frame='galactic') #the center of the image, used for cropping

desired_beam_size = 26.5*u.pc #I add this to the PI later. choose this based on the largest common beam size between the compared observations
#desired_beam_size = 36*u.pc #I add this to the PI later. choose this based on the largest common beam size between the compared observations
distance = 8.178*10**3*u.pc
Pointing_Information["distance"] = distance.to(u.Mpc) #in Mpc
Pointing_Information["target_image_rotation"]=0*u.deg #this is the rotation of the specific image, not the target. (use clockwise rotation angle)
Pointing_Information["target_inclination"]=0*u.deg
Pointing_Information["target_velocity"]=0*u.km/u.s #the speed its moving away from us
#Pointing_Information["target_velocity"]=250*u.km/u.s #the speed NGC253 is moving away from us
Pointing_Information["vaxis"]=0 #which axis is the velocity
Pointing_Information["desired_velocity_resolution"] = 5*u.km/u.s
ovs = 5#5 #how much do you desire to oversample the beam by
#desired_FOV = [900,2000]*u.pc# pc normal to the disk, across the disk
desired_FOV = [900,2000]*u.pc# pc normal to the disk, across the disk
Pointing_Information["Corresponding_Continuum"] = 'gc_850micron_dust.fits' # this is for the band 7, not for HCN J1-0 #The continuum image for this band
#                                                 '4.3pc_beam_CMZ_850um_Cont_140x800pc.fits' # this is the JCMT continuum for 850microns (the CO 3-2 and HCN, HCO 4-3)


print("done")

gc.collect








#This stuff will input automagically if the rest is correct

sc = SpectralCube.read("Spectral Cubes/"+Pointing_Information["Original_File_Name"],format="fits")
header = sc.header
print()
try:
    freq = header["RESTFREQ"]*u.Hz# assuming the header is in Hz
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFREQ"]#
    Pointing_Information['restfreq']=freq         
except:
    freq = header["RESTFRQ"]*u.Hz#
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFRQ"]#            
    Pointing_Information['restfreq']=freq 

######calculate teh beam size of the original image:

if (header['CUNIT1'].find("deg")!=-1):
    CUNIT = 1*u.degree
    Pointing_Information["CUNIT"]=CUNIT
else:
    print("The header should show CUNIT in degrees. If not, just fix this or write CUNIT = the unit it says in the header")
    del jhgasdhgjkahsdkgdfjhsgjgjsdhkfjghjd #this causes an error and stops execution 

try:

    # Get the beam size. If there is none, use the pixel size
    beam_major =  (header["BMAJ"]*CUNIT).to(u.arcsec) #degrees beam size -> arcsec
    beam_minor =  (header["BMIN"]*CUNIT).to(u.arcsec) #if these two are different, something went wrong in the reprojection 
    Cube_Information['Beam_Position_angle'] = header['bpa']*u.deg


except:

    cdelt_x = u.Quantity(str(np.abs(header['cdelt1']))+header['cunit1'])
    cdelt_y = u.Quantity(str(np.abs(header['cdelt2']))+header['cunit2'])
    if(cdelt_x>cdelt_y):
        beam_major=cdelt_x
        beam_minor=cdelt_y
    elif(cdelt_x<cdelt_y):
        beam_major=cdelt_y
        beam_minor=cdelt_x
    elif(cdelt_x==cdelt_y):
        beam_major=cdelt_x
        beam_minor=cdelt_x



Pointing_Information["original_BMAJ"]=beam_major
Pointing_Information["original_BMIN"]=beam_minor
Pointing_Information["original_BMAJ_pc"]=beam_major.to(u.rad)*Pointing_Information['distance']
Pointing_Information["original_BMIN_pc"]=beam_minor.to(u.rad)*Pointing_Information['distance']
Pointing_Information["desired_beam_size"] = desired_beam_size #ill put it in a circular beam at this size


#this accounts for elliptical beams:            
Pointing_Information['original_pixel_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix
Pointing_Information['original_pixel_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix
Pointing_Information['original_spatial_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']
Pointing_Information['original_spatial_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']#convert to pc using the distance

average_pixel=np.sqrt((abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix*(abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix)

Pointing_Information['original_beam_oversampling_MAJ'] = beam_major/average_pixel
Pointing_Information['original_beam_oversampling_MIN'] = beam_minor/average_pixel
Pointing_Information['desired_beam_oversampling'] = ovs
#Pointing_Information['orig_FOV']= Crop_Around_Center(sc,Pointing_Information['target_image_rotation'],Pointing_Information['center'],desired_FOV,Pointing_Information['distance'])[1] #returns the current fov
Pointing_Information['desired_FOV']=desired_FOV

######

Cube_Information = Update_Cube_Information(Pointing_Information,Pointing_Information["Original_File_Name"])


done



# 1. Reprojection

In [11]:
gc.collect()

Reproject_To_Region(Pointing_Information,Cube_Information,i_step=10,Cube_Name_Save='',Force_Origin=[True,[357,3],[-.6,.6]],partway=True)


# This is the cube name that they are being saved to unless you changed it inside the reproject functions definiton

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

gc.collect()

print("done")


begin step: 0 of 24
start beam convolution
0.0 0.0
0.03445275650355069 0.03445275650355069
convolve end

fov crop start 

Center: <SkyCoord (Galactic): (l, b) in deg
    (0., -0.)> Pixel center: 672 168 Pixel FOV: [1513, 3363]
cropping cube. rotation: 0.0 deg center: <SkyCoord (Galactic): (l, b) in deg
    (0., -0.)> crop to: [ 900. 2000.] pc
Cropped to  [ 900. 2000.] pc pixels 1 336 1 1056
check max SC value after crop: 0.5618081824524273 SC shape: (10, 335, 1055)
fov cropped

Force values of: [357, 3] [-0.6, 0.6] Giving pixel range of  161 32
Delete LBOUND if there is one in the header, if not this will pass the error:
"Keyword 'LBOUND1' not found."
------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_961702/3501929740.py", line 277, in Reproject_To_Region
    del reheader['LBOUND1']
  File "/home/ben/.local/lib/python3.8/site-packages/astropy/io/fits/header.py", line 239, in __delitem__
    raise KeyError(f"Keyword '{

# 1a. Splice the reprojected cubes back together

In [12]:
# First, splice the partwise cubes back together

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Splice_vels(Cube_Name_Load=Cube_Name_Save,istart=1)

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

gc.collect()

print("done")

Loaded 0_26.5pc_beam_cmz_CO_J1_0_high_res900.0x2000.0pc_reprojected.fits
local variable 'scW' referenced before assignment
------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_961702/3547181076.py", line 33, in Splice_vels
    scW = np.concatenate((scW[:],sc.hdu.data[:]),dtype = type(sc))
UnboundLocalError: local variable 'scW' referenced before assignment


UnboundLocalError: local variable 'scW' referenced before assignment

# 1b. Velocity reprojection

In [ ]:
Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Load the spliced cube

sc = SpectralCube.read(("Spectral Cubes/"+Cube_Name_Save)) 

# Reproject the spliced Cube's veloctiy axis

Repo_Velocity(sc,Cube_Name_Save,Cube_Information)

# Here is the new cube's name

Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

del sc

gc.collect()

# 1c. Fix repeated pixels

In [ ]:
# after the reprojection, there may be duplicated data from the fact that it just fills in stuff 
# expecting new data to be there. This fixes it by checking to see if it matches other data, then removes
# the repeated slices. in other words, it may be set to reproject to 0-500 km/s through the velocity channels
# but there may only be data between 100-400, so it will fill it in with copied data channels. This will check for those.



# Update to the current cube name

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Run the function to crop the nan values out of the cube

Remove_Repeated_Pixels(Cube_Name_Save)

# x. save cube information

In [ ]:
# Now that all the data reduction is concluded, save information about the new cubes

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Save = "Cropped_"+Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

Cube_Information_N = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

import pickle 

with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Cube_Information", 'wb') as file:
    pickle.dump(Cube_Information_N, file)

with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Pointing_Information", 'wb') as file:
    pickle.dump(Pointing_Information, file)

print(Cube_Information)

# xb. Noise matching

# Return an input cube matched to the given nosie

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Save = "Cropped_"+Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

Q  = SpectralCube.read("Spectral Cubes/"+Cube_Name_Save)

NM_Cube = Noise_matching(Input_Cube=Q,m=(.115*u.K))
    
    
Cube_Name_Save = "NM_"+Cube_Name_Save

NM_Cube.write("Spectral Cubes/"+Cube_Name_Save)
    


Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

import pickle 

with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Cube_Information", 'wb') as file:
    pickle.dump(Cube_Information, file)

with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Pointing_Information", 'wb') as file:
    pickle.dump(Pointing_Information, file)

print(Cube_Information)

In [45]:
"""depreaceated; ignore:
#Fix header
files = ["COGAL_deep_interp.fits"]
#files=["Pre-Smoothed.fits"]



sc = SpectralCube.read("Spectral Cubes/"+files[0])
f_header = SpectralCube.read("Spectral Cubes/"+"_cmz_CO_J3_2_Spliced_Smoothed_003-357.fits").header
Cube_Name = files[0]+"_Header_Fix.fits"
print(1)
sc.allow_huge_operations=True
sc = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s


#1.152712e+11 Hz

reheader = copy.deepcopy(sc.hdu.header) 


reheader_1 = astropy.io.fits.Header(cards=[], copy=False)
#reheader_1["RESTFRQ"] = 115.2712018 * 10**9 # Hz
#reheader_1["CUNIT3"] = "km/s"
#reheader_1["CRVAL3"] = float(reheader["CRVAL3"])*1000 #Changing from km/s to m/s because the original is in km/s but says its in m/s
#reheader_1["CRVAL1"] = 0
#reheader_1["CRPIX1"] = -1439.5
#reheader_1["LATPOLE"] = 0
#reheader_1["LONPOLE"] = 0


for key in reheader.keys():
    if key == "CRVAL1" or key == "CRPIX1" or key == "CUNIT3" or  key == "CRVAL3" or key == "LATPOLE" or key == "LONPOLE" or key =="MJDREF":
        #if key == "CRVAL1" or key == "CRPIX1" or key == "CUNIT3" or  key == "CRVAL3":
        print("not adding:",key,reheader[key])
        #if key == "CUNIT3" or  key == "CRVAL3":
        #if key == "nothing":
        #continue
    try:
        if key == "CDELT3":
            #reheader_1[key] = 1.3 # The initial header has incorrectly labelled the velocity resolution
            reheader_1[key] = 1.3 # The initial header has incorrectly labelled the velocity resolution
            print("Fixing velocity res to 1.3 km/s" )
            #elif key == "CRPIX1":
            #reheader_1["CRPIX1"] = -1439.5
            #elif key == "CRVAL1":
            #reheader_1["CRVAL1"] = 0
        elif key == "CRVAL3":
            reheader_1["CRVAL3"] = float(reheader["CRVAL3"])*1000 #Changing from km/s to m/s because the original is in km/s but says its in m/s
        elif key == "CUNIT3":
            reheader_1["CUNIT3"] = "km/s"
            '''
            elif key == "CTYPE1":
                reheader_1[key] = "GLON"
            elif key == "CTYPE2":
                reheader_1[key] = "GLAT"
            '''


        elif key.find("COMMENT")!=-1:
            pass
        else:
            reheader_1[key] = reheader[key]
            print(key,reheader[key])

    except:
        
        print("\n\n\nCannot add:            \n",key,reheader[key],"\n\n\n")
        

reheader_1["RESTFRQ"] = 115.2712018 * 10**9 # Hz

#reheader["RESTFRQ"] = 115.2712018 * 10**9 # Hz
#reheader["CUNIT3"] = "km/s"
#reheader["CRVAL1"] = 360
#print(reheader.cards)
print(WCS(reheader),reheader,WCS(reheader_1),reheader_1,reheader_1.cards)

#print(np.shape(sc))
#print(sc.hdu.header)
#print(reheader)
#print(sc.wcs)


print(WCS(reheader_1).all_pix2world([[0,0,0],[0,0,0]],0))
# This only matters for formatting reasons:
def duh(lol):
    gp = np.where(lol!=np.nan)
    #lol[gp]=True
    return lol # Anywhere that has data will be unmasked
Full_Mask = LazyMask(function = duh,data = sc.mask, wcs = WCS(reheader_1))
#www = LazyMask(function = duh,data = sc.hdu.data, wcs = WCS(reheader))
www = LazyMask(function = duh,data = sc.hdu.data, wcs = WCS(reheader))

header_test = astropy.io.fits.Header(cards=[], copy=False)


for key in reheader.keys():

    if key == "CDELT3":
        #reheader_1[key] = 1.3 # The initial header has incorrectly labelled the velocity resolution
        header_test[key] = 1.3 # The initial header has incorrectly labelled the velocity resolution
        print("Fixing velocity res to 1.3 km/s" )
        '''
        elif key == "CRPIX1":
            header_test["CRPIX1"] = -1439.5
        elif key == "CRVAL1":
            header_test["CRVAL1"] = 0
        elif key == "CRVAL3":
            header_test["CRVAL3"] = float(reheader["CRVAL3"])*1000 #Changing from km/s to m/s because the original is in km/s but says its in m/s
        elif key == "CUNIT3":
            header_test["CUNIT3"] = "km/s"
        elif key == "CRVAL2":
            header_test[key] = 0
        elif key == "CRPIX2":
            header_test[key] = 0
        elif key == "CDELT2":
            header_test[key] = reheader[key]
        elif key == "CDELT1":
            header_test[key] = reheader[key]
        '''
    elif key == "CRPIX1":
        header_test["CRPIX1"] = reheader[key]
    elif key == "CRVAL1":
        header_test["CRVAL1"] = reheader[key]
    elif key == "CRVAL3":
        header_test["CRVAL3"] =reheader[key]
    elif key == "CUNIT3":
        header_test["CUNIT3"] = reheader[key]
    elif key == "CRVAL2":
        header_test[key] = reheader[key]
    elif key == "CRPIX2":
        header_test[key] = reheader[key]
    elif key == "CDELT2":
        header_test[key] = reheader[key]
    elif key == "CDELT1":
        header_test[key] = reheader[key]
    header_test["NAXIS"]=3
    header_test["NAXIS1"]=2881
    header_test["NAXIS2"]=81
    header_test["NAXIS3"]=493
    header_test["BUNIT"]="K"
    header_test["WCSAXES"]=3
    header_test["RESTFRQ"] = 115.2712018 * 10**9 # Hz
    header_test["CTYPE1"]='GLON-CAR'
    header_test["CTYPE2"]='GLAT-CAR'
    #header_test["CTYPE1"]='RA'
    #header_test["CTYPE2"]='DEC'
    header_test["CTYPE3"]='VOPT'
    
    #('CTYPE1', 'GLON-CAR', '')
    #('CTYPE2', 'GLAT-CAR', '')
    #('CTYPE3', 'VOPT', '')
    

print("header test",header_test)
ttttttt = fits.PrimaryHDU(data=sc.hdu.data, header=header_test)
ttttttt.writeto("Spectral Cubes/wwwwwwwwwww.fits", overwrite=True)
print("-"*700)
print(ttttttt)
hdu = fits.PrimaryHDU(data=sc.hdu.data, header=reheader_1)
hdu.writeto("Spectral Cubes/tttttttttttt.fits", overwrite=True)

wwwwwwwww = fits.PrimaryHDU(data=sc.hdu.data, header=f_header)
wwwwwwwww.writeto("Spectral Cubes/f_header.fits", overwrite=True)
aaaa=SpectralCube.read("Spectral Cubes/tttttttttttt.fits")

new = SpectralCube(data=sc.hdu.data,wcs =WCS(reheader_1),mask=Full_Mask)
new.write("Spectral Cubes/aaaaaaaaaaaaaaaaaaaa.fits")
print(aaaa.latitude_extrema)
print(aaaa.mask[0][0][0])
print('\n\n\n\n\n\n g')
#print(sc.mask[0][0][0])
print(sc.hdu.data[0][0][0])
print(www)
print(www[0][0][0])
print('\n\n\n\n\n g')




#new = SpectralCube(data=sc.hdu.data,wcs =WCS(reheader),mask=sc.mask)
#new = SpectralCube(data=sc.hdu.data,wcs =WCS(reheader_1),mask=sc.mask)
new = SpectralCube(data=sc.hdu.data,wcs =WCS(reheader_1),mask=Full_Mask)

new.allow_huge_operations=True
print(new[0][0][0],new.mask[0][0][0])
new=new*sc[0][0][0].unit
print(2)
del sc
sc=new
print(3)
print(sc.wcs)
print(sc.wcs.to_header())
del new

sc.write("Spectral Cubes/"+Cube_Name,overwrite=True)
print(sc.header)

print(4)
print("done")

"""

1
SIMPLE True
BITPIX -32
NAXIS 3
NAXIS1 2881
NAXIS2 81
NAXIS3 493
DATAMIN -1.0
DATAMAX 20.0
BUNIT K
WCSAXES 3
not adding: CRPIX1 1.0
CRPIX1 1.0
CRPIX2 41.0
CRPIX3 1.0
CDELT1 -0.125
CDELT2 0.125
Fixing velocity res to 1.3 km/s
CUNIT1 deg
CUNIT2 deg
not adding: CUNIT3 km s-1
CTYPE1 GLON-CAR
CTYPE2 GLAT-CAR
CTYPE3 VOPT
not adding: CRVAL1 180.0
CRVAL1 180.0
CRVAL2 0.0
not adding: CRVAL3 -0.31980000000000003
not adding: LONPOLE 0.0
LONPOLE 0.0
not adding: LATPOLE 90.0
LATPOLE 90.0
not adding: MJDREF 0.0
MJDREF 0.0
SPECSYS LSRK
WCS Keywords

Number of WCS axes: 3
CTYPE : 'GLON-CAR'  'GLAT-CAR'  'VOPT'  
CRVAL : 180.0  0.0  -319.79999999999995  
CRPIX : 1.0  41.0  1.0  
PC1_1 PC1_2 PC1_3  : 1.0  0.0  0.0  
PC2_1 PC2_2 PC2_3  : 0.0  1.0  0.0  
PC3_1 PC3_2 PC3_3  : 0.0  0.0  1.0  
CDELT : -0.125  0.125  1.3  
NAXIS : 2881  81  493 SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -32 / array data type                                NAXIS 

InvalidSubimageSpecificationError: ERROR 12 in wcssub() at line 898 of file cextern/wcslib/C/wcs.c:
Invalid subimage specification.


In [31]:
"""depreceated; ignore:
#Fix header (This one works)
files = ["COGAL_deep_interp.fits"]
#files=["Pre-Smoothed.fits"]



sc = SpectralCube.read("Spectral Cubes/"+files[0])  
Cube_Name = files[0]+"_Header_Fix.fits"
print(1)
sc.allow_huge_operations=True
sc = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio").to(u.K) # Change units from Hz to km/s

reheader = sc.header
reheader["RESTFRQ"] = 115.2712018 * 10**9 # Hz

new = SpectralCube(data=sc.hdu.data,wcs =WCS(reheader),mask=sc.mask)
new.allow_huge_operations=True
new=new*sc[0][0][0].unit
print(2)
del sc
sc=new
print(3)
del new

sc.write("Spectral Cubes/"+Cube_Name,overwrite=True)
print(4)
print("done")
"""

1
2
3
4
done
